In [ ]:
from entsoe import EntsoePandasClient
import pandas as pd
import numpy as np
import plotly.express as px
import datetime
import plotly.graph_objects as go
import random
import warnings

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None 

client = EntsoePandasClient(api_key='API-KEY')

start = pd.Timestamp('20230101', tz='Europe/Brussels')
end = pd.Timestamp('20240101', tz='Europe/Brussels')
country_code = 'DE'  # Germany
# country_code_from = 'FR'  # France
# country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A51'

In [ ]:
def convert_time(df):

    if " - " in df.index:
        # Split the time interval into start and end times
        df["Start Time"] = df.index.str.split(" - ", expand=True)[0]
        # Convert the "Start Time" to datetime
        df["Start Time"] = pd.to_datetime(df["Start Time"], format="mixed")
        df.index = df["Start Time"]
    else:
        # Convert the time data to datetime
        df.index = pd.to_datetime(df.index, format="mixed")

    return df

In [ ]:

def display_stats(df_read):


    for column in df_read.columns:
        # Compute descriptive statistics for the current column
        stats = df_read[column].describe()

        # Construct the summary text with markdown formatting
        summary_text = f"""
        ### Statistics for {column}
        **Count**: {stats['count']} data points.\n
        **Mean**: The average is {stats['mean']:.2f}.\n
        **Standard Deviation**: The standard deviation is {stats['std']:.2f}, which indicates variability.\n
        **Minimum**: The smallest observed value is {stats['min']:.2f}.\n
        **25th Percentile**: 25% of the values are {stats['25%']:.2f} or less.\n
        **50th Percentile (Median)**: The median value is {stats['50%']:.2f}.\n
        **75th Percentile**: 75% of the values are {stats['75%']:.2f} or less.\n
        **Maximum**: The largest observed value is {stats['max']:.2f}.
        """

        # Display the summary
        print(summary_text)

In [354]:
def filter_data(df,start_date,end_date):
    
    # Filter the DataFrame for the selected date range
    df_days_of_interest = df.loc[
        start_date.strftime("%Y-%m-%d") : end_date.strftime("%Y-%m-%d")
    ]
    return df_days_of_interest    

In [ ]:
def visualize_data_by_date_range(df_read):
    
    # Create a line plot for each column in the DataFrame
    fig2 = go.Figure()
    for column in df_read.columns:
        fig2.add_trace(
            go.Scatter(
                x=df_read.index, y=df_read[column], name=column
            )
        )
    # Display the figure
    fig2.show()


In [ ]:
def visualise_data(df_read):

    fig3 = go.Figure()
    fig4 = go.Figure()
    fig5 = go.Figure()

    for column in df_read.columns:
        # Generate a random color
        random_color = 'rgb(%d, %d, %d)' % (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        # Resample and calculate mean for each day
        daily_mean = df_read[column].resample('D').mean()

        # Resample for monthly peak and mean
        monthly_peak = df_read[column].resample('M').max()
        monthly_mean = df_read[column].resample('M').sum()
        monthly_change = monthly_mean.diff()

        # Add traces for daily mean, monthly peak, and monthly change
        fig3.add_trace(go.Scatter(x=daily_mean.index, y=daily_mean, name=column, line=dict(color=random_color)))
        fig4.add_trace(go.Bar(x=monthly_peak.index.strftime('%B'), y=monthly_peak, name=column, marker_color=random_color))
        fig5.add_trace(go.Bar(x=monthly_change.index, y=monthly_change, name=f'Monthly Change - {column}', marker_color=random_color))

    # Update layout for each figure
    fig3.update_layout(title={'text': "Daily Load "})
    fig4.update_layout(title='Monthly Peak Values', xaxis_title='Month', yaxis_title='Peak Value', barmode='group')
    fig5.update_layout(title='Monthly Load Change', xaxis_title='Date', yaxis_title='Change in Aggregate Load', barmode='group')

    fig3.show()
    fig4.show()
    fig5.show()

In [357]:
df_read=client.query_generation(country_code, start=start, end=end, psr_type=None)
df_read = df_read.drop('Actual Consumption', axis = 1, level = 1)
df_read = df_read.droplevel(1, axis=1) 
df_read = convert_time(df_read)

In [ ]:
visualise_data(df_read)
#display_stats(df) 


In [ ]:
# Select a range of dates
start_date = datetime.date(2023, 6, 1)
end_date = datetime.date(2023, 7, 1)
df_filtered = filter_data(df_read,start_date,end_date)

In [387]:
visualize_data_by_date_range(df_filtered)
df_filtered['hour'] = df_filtered.index.hour
df_s = df_filtered.groupby(['hour']).mean().transpose()
(df_s
 .fillna(0)
 .style
     .format(precision=0, thousands=',')
     .background_gradient(cmap='Reds', axis=1)
)

hour,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
Biomass,"3,989","3,941","3,922","3,911","3,936","4,008","4,197","4,285","4,257","4,131","4,006","3,946","3,894","3,869","3,858","3,874","3,931","4,022","4,201","4,341","4,390","4,355","4,251","4,122"
Fossil Brown coal/Lignite,"8,135","7,845","7,696","7,616","7,623","7,791","8,032","7,985","7,761","7,331","6,832","6,413","6,289","6,147","6,132","6,262","6,638","7,391","8,192","8,564","8,724","8,798","8,747","8,506"
Fossil Coal-derived gas,569,546,550,545,545,542,543,539,541,545,554,555,547,551,550,548,549,552,557,554,553,550,557,555
Fossil Gas,"6,066","5,494","5,281","5,165","5,171","5,403","5,898","6,108","6,036","5,627","4,979","4,448","4,223","4,154","4,176","4,133","4,365","5,148","6,474","7,284","7,479","7,536","7,424","6,956"
Fossil Hard coal,"2,284","1,985","1,845","1,824","1,856","2,047","2,364","2,453","2,448","2,269","2,045","1,883","1,836","1,784","1,825","1,864","2,014","2,320","2,802","3,106","3,142","3,063","2,936","2,669"
Fossil Oil,355,353,353,353,353,353,352,356,358,358,354,355,355,358,361,363,364,367,371,365,362,365,361,356
Geothermal,20,20,20,21,21,21,21,20,19,18,17,17,16,16,15,15,15,15,16,16,17,18,18,19
Hydro Pumped Storage,442,246,166,173,182,213,"1,219","2,225","1,821",869,428,257,172,128,118,134,219,520,"1,241","2,941","4,078","3,555","2,530","1,162"
Hydro Run-of-river and poundage,"1,835","1,837","1,834","1,825","1,819","1,818","1,823","1,832","1,844","1,861","1,879","1,883","1,877","1,850","1,847","1,853","1,855","1,863","1,850","1,839","1,833","1,832","1,832","1,829"
Hydro Water Reservoir,209,174,175,151,155,171,332,389,328,257,217,183,197,166,164,181,210,245,312,384,467,429,377,261
